In [1]:
%matplotlib inline
import pandas as pd
import os
import networkx as nx
import igraph as ig
import numpy as np
import louvain
import matplotlib.pyplot as plt

In [2]:
filepath = '/media/sf_VBox_Shared/CaseLaw/2018-01-29-lido/derived/'

In [3]:
links_df = pd.read_csv(os.path.join(filepath, 'case_to_article_title_links.csv'))

In [4]:
case_to_case_links = pd.read_csv(os.path.join(filepath, 'case_to_case_lx_links.csv'))

In [5]:
art_nodes = pd.read_csv(os.path.join(filepath, 'article_title_nodes.csv'))
case_nodes = pd.read_csv(os.path.join(filepath, 'case_nodes_simple.csv'))

In [6]:
case_nodes['id'] = case_nodes['lido_id']
case_nodes['label'] = case_nodes['ecli'].str.split(':').map(lambda l: '{}:{}'.format(l[2], l[3]) if type(l)==list else l)

In [7]:
art_nodes.head()

,id,label,book
0,"Loi concernant les Mines, les Minières et les ...","Loi concernant les Mines, les Minières et les ...",Loi concernant les Mines
1,"Loi concernant les Mines, les Minières et les ...","Loi concernant les Mines, les Minières et les ...",Loi concernant les Mines
2,"Loi concernant les Mines, les Minières et les ...","Loi concernant les Mines, les Minières et les ...",Loi concernant les Mines
3,"Loi concernant les Mines, les Minières et les ...","Loi concernant les Mines, les Minières et les ...",Loi concernant les Mines
4,"Loi concernant les Mines, les Minières et les ...","Loi concernant les Mines, les Minières et les ...",Loi concernant les Mines


In [8]:
case_nodes.head()

,ecli,lido_id,country,court,year,id,label
0,ECLI:NL:GHSGR:2010:BM2753,http://linkeddata.overheid.nl/terms/jurisprude...,NL,GHSGR,2010.0,http://linkeddata.overheid.nl/terms/jurisprude...,GHSGR:2010
1,ECLI:NL:RBHAA:2007:BC0491,http://linkeddata.overheid.nl/terms/jurisprude...,NL,RBHAA,2007.0,http://linkeddata.overheid.nl/terms/jurisprude...,RBHAA:2007
2,ECLI:NL:RBMAA:2007:BC1654,http://linkeddata.overheid.nl/terms/jurisprude...,NL,RBMAA,2007.0,http://linkeddata.overheid.nl/terms/jurisprude...,RBMAA:2007
3,ECLI:NL:CRVB:2016:4389,http://linkeddata.overheid.nl/terms/jurisprude...,NL,CRVB,2016.0,http://linkeddata.overheid.nl/terms/jurisprude...,CRVB:2016
4,ECLI:NL:RBLIM:2014:11172,http://linkeddata.overheid.nl/terms/jurisprude...,NL,RBLIM,2014.0,http://linkeddata.overheid.nl/terms/jurisprude...,RBLIM:2014


In [9]:
seed_articles = art_nodes[art_nodes.label.str.match('Burgerlijk Wetboek Boek 7, Artikel ((611)|(658))', case=False)]
seed_articles

,id,label,book
61134,"Burgerlijk Wetboek Boek 7, Artikel 611a","Burgerlijk Wetboek Boek 7, Artikel 611a",Burgerlijk Wetboek Boek 7
61150,"Burgerlijk Wetboek Boek 7, Artikel 658c","Burgerlijk Wetboek Boek 7, Artikel 658c",Burgerlijk Wetboek Boek 7
61393,"Burgerlijk Wetboek Boek 7, Artikel 611","Burgerlijk Wetboek Boek 7, Artikel 611",Burgerlijk Wetboek Boek 7
61442,"Burgerlijk Wetboek Boek 7, Artikel 658","Burgerlijk Wetboek Boek 7, Artikel 658",Burgerlijk Wetboek Boek 7
61502,"Burgerlijk Wetboek Boek 7, Artikel 658a","Burgerlijk Wetboek Boek 7, Artikel 658a",Burgerlijk Wetboek Boek 7
61649,"Burgerlijk Wetboek Boek 7, Artikel 658b","Burgerlijk Wetboek Boek 7, Artikel 658b",Burgerlijk Wetboek Boek 7


In [10]:
linking_cases = links_df[links_df.target.isin(seed_articles.id)]['source'].unique()
len(linking_cases)

2842

In [12]:
linking_cases[:10]

array([ 'http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CRVB:2001:AL3230',
       'http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CRVB:2002:AE3431',
       'http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CRVB:2002:AE3622',
       'http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CRVB:2002:AE7111',
       'http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CRVB:2002:AE8899',
       'http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CRVB:2002:AE8902',
       'http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CRVB:2002:AE8914',
       'http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CRVB:2002:AE8964',
       'http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CRVB:2002:AE8965',
       'http://linkeddata.overheid.nl/terms/jurisprudentie/id/ECLI:NL:CRVB:2002:AF1843'], dtype=object)

In [10]:
links_sub = links_df[links_df.source.isin(linking_cases)]
links_sub.shape

(11962, 2)

In [11]:
case_to_case_sub = case_to_case_links[case_to_case_links.source.isin(linking_cases)&case_to_case_links.target.isin(linking_cases)]
case_to_case_sub.shape

(1608, 3)

In [12]:
art_nodes_sub = art_nodes[art_nodes.id.isin(links_sub.target.unique())]
case_nodes_sub = case_nodes[case_nodes.lido_id.isin(linking_cases)]
print(art_nodes_sub.shape, case_nodes_sub.shape)

(1378, 3) (1596, 7)


In [11]:
case_nodes_sub['node_type'] = 0
art_nodes_sub['node_type'] = 1

In [22]:
links_sub['link_type'] = 'case-article'
case_to_case_sub['link_type'] = 'case-case'

In [23]:
case_to_case_sub.to_csv(os.path.join(filepath, 'subnetwork', 'case-to-case-links.csv'), index=False)
links_sub.to_csv(os.path.join(filepath, 'subnetwork', 'case-to-article-links.csv'), index=False)
art_nodes_sub.to_csv(os.path.join(filepath, 'subnetwork', 'article-nodes.csv'), index=False)
case_nodes_sub.to_csv(os.path.join(filepath, 'subnetwork', 'case-nodes-sub.csv'), index=False)

In [8]:
case_to_case_sub = pd.read_csv(os.path.join(filepath, 'subnetwork', 'case-to-case-links.csv'))
links_sub = pd.read_csv(os.path.join(filepath, 'subnetwork', 'case-to-article-links.csv'))
art_nodes_sub = pd.read_csv(os.path.join(filepath, 'subnetwork', 'article-nodes.csv'))
case_nodes_sub = pd.read_csv(os.path.join(filepath, 'subnetwork', 'case-nodes-sub.csv'))

In [8]:
case_nodes_sub.head()

,ecli,lido_id,country,court,year,id,label,node_type
0,ECLI:NL:CRVB:2016:2999,http://linkeddata.overheid.nl/terms/jurisprude...,NL,CRVB,2016.0,http://linkeddata.overheid.nl/terms/jurisprude...,CRVB:2016,0
1,ECLI:NL:RBROT:2013:BZ2568,http://linkeddata.overheid.nl/terms/jurisprude...,NL,RBROT,2013.0,http://linkeddata.overheid.nl/terms/jurisprude...,RBROT:2013,0
2,ECLI:NL:GHAMS:2011:BU6149,http://linkeddata.overheid.nl/terms/jurisprude...,NL,GHAMS,2011.0,http://linkeddata.overheid.nl/terms/jurisprude...,GHAMS:2011,0
3,ECLI:NL:GHARL:2014:6359,http://linkeddata.overheid.nl/terms/jurisprude...,NL,GHARL,2014.0,http://linkeddata.overheid.nl/terms/jurisprude...,GHARL:2014,0
4,ECLI:NL:RBNHO:2017:5966,http://linkeddata.overheid.nl/terms/jurisprude...,NL,RBNHO,2017.0,http://linkeddata.overheid.nl/terms/jurisprude...,RBNHO:2017,0


In [10]:
case_to_case_links.head()

,link_id,source,target
0,http://localhost:8984/bigdata/namespace/Regell...,http://linkeddata.overheid.nl/terms/jurisprude...,http://linkeddata.overheid.nl/terms/jurisprude...
1,http://localhost:8984/bigdata/namespace/Regell...,http://linkeddata.overheid.nl/terms/jurisprude...,http://linkeddata.overheid.nl/terms/jurisprude...
2,http://localhost:8984/bigdata/namespace/Regell...,http://linkeddata.overheid.nl/terms/jurisprude...,http://linkeddata.overheid.nl/terms/jurisprude...
3,http://localhost:8984/bigdata/namespace/Regell...,http://linkeddata.overheid.nl/terms/jurisprude...,http://linkeddata.overheid.nl/terms/jurisprude...
4,http://localhost:8984/bigdata/namespace/Regell...,http://linkeddata.overheid.nl/terms/jurisprude...,http://linkeddata.overheid.nl/terms/jurisprude...


In [9]:
# Make smaller subnetwork
case_nodes_hr = case_nodes_sub[case_nodes_sub['court']=='HR']
links_sub_hr = links_sub[links_sub.source.isin(case_nodes_hr.id)]
art_nodes_hr = art_nodes_sub[art_nodes_sub.id.isin(links_sub_hr.target.unique())]
case_to_case_hr = case_to_case_sub[case_to_case_sub.source.isin(case_nodes_hr.id)&case_to_case_sub.target.isin(case_nodes_hr.id)]
case_nodes_hr.shape, links_sub_hr.shape, art_nodes_hr.shape, case_to_case_hr.shape

((118, 8), (408, 3), (140, 4), (74, 4))

In [26]:
case_to_case_hr.to_csv(os.path.join(filepath, 'subnetwork', 'case-to-case-links-hr.csv'), index=False)
links_sub_hr.to_csv(os.path.join(filepath, 'subnetwork', 'case-to-article-links-hr.csv'), index=False)
art_nodes_hr.to_csv(os.path.join(filepath, 'subnetwork', 'article-nodes-hr.csv'), index=False)
case_nodes_hr.to_csv(os.path.join(filepath, 'subnetwork', 'case-nodes-hr.csv'), index=False)

In [16]:
# Extend with all cases citing the seed cases
case_to_case_sub_ext = case_to_case_links[case_to_case_links.target.isin(case_nodes_sub.lido_id)]
linking_cases = set(case_to_case_sub_ext.source).union(set(case_to_case_sub_ext.target))
case_nodes_sub_ext = case_nodes[case_nodes.lido_id.isin(linking_cases)]
print(case_to_case_sub_ext.shape, case_nodes_sub_ext.shape)

(2795, 3) (2086, 7)


In [18]:
# Make smaller subnetwork
case_nodes_ext_hr = case_nodes_sub_ext[case_nodes_sub_ext['court']=='HR']
case_to_case_ext_hr = case_to_case_sub_ext[case_to_case_sub_ext.source.isin(case_nodes_ext_hr.id)&case_to_case_sub_ext.target.isin(case_nodes_ext_hr.id)]
case_nodes_ext_hr.shape, case_to_case_ext_hr.shape

((130, 7), (84, 3))

In [21]:
case_to_case_sub_ext.to_csv(os.path.join(filepath, 'subnetwork', 'case-to-case-links-ext.csv'), index=False)
case_nodes_sub_ext.to_csv(os.path.join(filepath, 'subnetwork', 'case-nodes-ext.csv'), index=False)

case_to_case_ext_hr.to_csv(os.path.join(filepath, 'subnetwork', 'case-to-case-links-ext-hr.csv'), index=False)
case_nodes_ext_hr.to_csv(os.path.join(filepath, 'subnetwork', 'case-nodes-ext-hr.csv'), index=False)

In [41]:
## cocitation network
import scipy.sparse
links_series = links_sub.groupby(['source', 'target']).nunique()['source']
mat_sparse = scipy.sparse.coo_matrix(
                (links_series.values, 
                 (links_series.index.labels[0], links_series.index.labels[1])))

In [47]:
case_ids = links_series.index.levels[0]
article_ids = links_series.index.levels[1]
print(len(case_ids), len(article_ids))

2842 1378


In [48]:
mat_cocitation = mat_sparse.T.dot(mat_sparse)
mat_cocitation.shape

(1378, 1378)

In [49]:
x_index, y_index, data = scipy.sparse.find(mat_cocitation)
df_cocitation = pd.DataFrame({'source':article_ids[x_index],
             'target': article_ids[y_index],
             'weight': data})
df_cocitation.shape

(33332, 3)

In [50]:
df_cocitation = df_cocitation[df_cocitation['source']<df_cocitation['target']]
df_cocitation.shape

(15977, 3)

In [52]:
df_cocitation_min5 = df_cocitation[df_cocitation.weight>=5]
df_cocitation_min5.shape

(997, 3)

In [54]:
df_cocitation_min5.to_csv(os.path.join(filepath, 'subnetwork', 'legislation-cociation-min5.csv'), index=False)

## enrich
Use caselawnet for these nodes

In [14]:
import caselawnet

[nltk_data] Downloading package punkt to /home/dafne/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [15]:
links_dict = [{'source': d['source'].split('/')[-1], 'target': d['target'].split('/')[-1]} for d in case_to_case_sub.to_dict(orient='records')]

In [16]:
enrich_again = False

if enrich_again:
    nodes_rich_list = caselawnet.enrich_eclis([s.split('/')[-1] for s in case_nodes_sub.id ], rootpath='/media/sf_VBox_Shared/CaseLaw/OpenDataUitspraken/')
    nodes_rich = pd.DataFrame(nodes_rich_list)
    nodes_rich.to_csv(os.path.join(filepath, 'subnetwork', 'nodes-cases-rich.csv'), index=False)
else:
    nodes_rich = pd.read_csv(os.path.join(filepath, 'subnetwork', 'nodes-cases-rich.csv')).fillna('')
    nodes_rich_list = nodes_rich.to_dict(orient='records')

In [17]:
nodes_rich.head()

,abstract,articles,count_annotation,count_version,creator,date,ecli,id,subject,title,year
0,,,,,,,ECLI:NL:PHR:2017:511,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,,,
1,,,,,,,ECLI:NL:GHSHE:2015:4088,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,,,
2,11 maart 2005 Eerste Kamer Nr. C03/289HR JMH H...,[],5,9,http://standaarden.overheid.nl/owms/terms/Hoge...,2005-03-11,ECLI:NL:HR:2005:AR6657,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,http://psi.rechtspraak.nl/rechtsgebied#civielR...,"ECLI:NL:HR:2005:AR6657 Hoge Raad , 11-03-2005 ...",2005
3,Ongeval ambtenaar; politieagent; afbakening be...,[],0,2,http://standaarden.overheid.nl/owms/terms/Rech...,2009-07-22,ECLI:NL:RBROT:2009:BJ5463,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,http://psi.rechtspraak.nl/rechtsgebied#civielR...,ECLI:NL:RBROT:2009:BJ5463 Rechtbank Rotterdam ...,2009
4,Geen schending zorgplicht ex art. 7:658 BW van...,"['7:658 bw', '7:611 bw', 'Burgerlijk Wetboek B...",0,6,http://standaarden.overheid.nl/owms/terms/Gere...,2011-10-04,ECLI:NL:GHAMS:2011:BU6935,http://deeplink.rechtspraak.nl/uitspraak?id=EC...,http://psi.rechtspraak.nl/rechtsgebied#civielR...,ECLI:NL:GHAMS:2011:BU6935 Gerechtshof Amsterda...,2011


In [18]:
links_list = caselawnet.enrich_links(links_dict)

In [19]:
nodes_network, links_list = caselawnet.get_network(nodes_rich_list, links_list)

In [20]:
caselawnet.to_sigma_json(nodes_network, links_list, 'Employer liability', os.path.join(filepath, 'subnetwork', 'network.json'))

In [21]:
nodes_network_df = pd.DataFrame(nodes_network)
nodes_network_df.community.value_counts().head()

1541    1
1023    1
164     1
1140    1
1085    1
Name: community, dtype: int64

In [22]:
graph = caselawnet.network_analysis.get_network(nodes_rich_list, links_list)

In [23]:
len(graph)

2577

In [24]:
graph = nx.readwrite.json_graph.node_link_graph({'nodes': nodes_rich_list, 'links': links_list},
                                       directed=True, multigraph=False)

In [25]:
graph.number_of_nodes(), graph.number_of_edges()

(1596, 1608)

In [26]:
def add_network_statistics(nodes, links):
    if len(nodes)==0:
        return nodes
    graph = get_network(nodes, links)
    degree = nx.degree(graph)
    if max(dict(degree).values()) > 0:
        hubs, authorities = get_hits(graph)
        statistics = {
            'degree': degree,
            'in_degree': graph.in_degree(),
            'out_degree': graph.out_degree(),

            'degree_centrality': nx.degree_centrality(graph),
            'in_degree_centrality': nx.in_degree_centrality(graph),
            'out_degree_centrality': nx.out_degree_centrality(graph),
            'betweenness_centrality': nx.betweenness_centrality(graph),
            'closeness_centrality': nx.closeness_centrality(graph),
            'pagerank': get_pagerank(graph),
            'hubs': hubs,
            'authorities': authorities
        }
    else:
        statistics = {}

    # for relative in-degree we sort on date
    derive_date = lambda k: k['date'] if k['date']!='' else '{}-01-01'.format(k['year'])
    nodes.sort(key=derive_date, reverse=True)
    for i, node in enumerate(nodes):
        nodeid = node['id']
        for var in statistics.keys():
            node[var] = statistics[var][nodeid]
        if 'in_degree' in node:
            node['rel_in_degree'] = node['in_degree'] / float(max(i, 1))
    get_community(graph, nodes)
    return nodes

In [27]:
case_nodes_sub.court.value_counts()

PHR        225
GHSHE      132
HR         118
CRVB        85
GHAMS       84
GHSGR       83
RBROT       71
GHARL       69
RBUTR       66
RBMNE       64
RBAMS       59
RBARN       45
GHARN       42
GHLEE       42
GHDHA       39
RBNNE       25
RBMAA       25
RBHAA       24
RBNHO       24
RBSGR       23
RBGEL       23
RBLEE       22
RBDHA       18
RBLIM       17
RBOBR       15
RBZLY       15
RBMID       14
RBSHE       13
RBASS       12
RBALM       12
RBZWB       11
RBZUT       10
RBBRE        9
RBOVE        9
RBDOR        9
RBGRO        8
RBALK        8
RBONE        3
KTGMID       2
RBROE        2
KTGROT       2
KTGAMS       2
OGHACMB      1
KTGHAR       1
KTGAAR       1
RBZWO        1
KTGDEL       1
KTGHAA       1
KTGTIE       1
KTGALM       1
KTGBRI       1
KTGBRE       1
OGHNAA       1
KTGSCH       1
OGEAC        1
KTGUTR       1
KTGBEE       1
Name: court, dtype: int64

In [28]:
art_nodes_sub.book.value_counts()

Wetboek van Burgerlijke Rechtsvordering (geldt in geval van digitaal procederen)                                                              179
Burgerlijk Wetboek Boek 7                                                                                                                     176
Burgerlijk Wetboek Boek 6                                                                                                                     102
Algemene wet bestuursrecht                                                                                                                     79
Arbeidsomstandighedenbesluit                                                                                                                   52
Burgerlijk Wetboek Boek 3                                                                                                                      51
Wetboek van Strafrecht                                                                                                      